In [1]:
install.packages("DBI", repos='http://cran.us.r-project.org')

In [2]:
install.packages("RSQLite", repos='http://cran.us.r-project.org')

In [4]:
library('RSQLite')

In [5]:
library(dplyr)
soccer_db <- src_sqlite("database.sqlite")

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [6]:
soccer_db

src:  sqlite 3.11.1 [database.sqlite]
tbls: Country, League, Match, Player, Player_Attributes, sqlite_sequence, Team,
  Team_Attributes

In [4]:
team <- tbl(soccer_db, "Team")
player <- tbl(soccer_db, "Player")

In [5]:
team

Source:   query [?? x 5]
Database: sqlite 3.11.1 [database.sqlite]

      id team_api_id team_fifa_api_id    team_long_name team_short_name
   <int>       <int>            <int>             <chr>           <chr>
1      1        9987              673          KRC Genk             GEN
2      2        9993              675      Beerschot AC             BAC
3      3       10000            15005  SV Zulte-Waregem             ZUL
4      4        9994             2007  Sporting Lokeren             LOK
5      5        9984             1750 KSV Cercle Brugge             CEB
6      6        8635              229    RSC Anderlecht             AND
7      7        9991              674          KAA Gent             GEN
8      8        9998             1747         RAEC Mons             MON
9      9        7947               NA     FCV Dender EH             DEN
10    10        9985              232 Standard de Liège             STL
# ... with more rows

In [6]:
player

Source:   query [?? x 7]
Database: sqlite 3.11.1 [database.sqlite]

      id player_api_id        player_name player_fifa_api_id
   <int>         <int>              <chr>              <int>
1      1        505942 Aaron Appindangoye             218353
2      2        155782    Aaron Cresswell             189615
3      3        162549        Aaron Doran             186170
4      4         30572      Aaron Galindo             140161
5      5         23780       Aaron Hughes              17725
6      6         27316         Aaron Hunt             158138
7      7        564793         Aaron Kuhl             221280
8      8         30895       Aaron Lennon             152747
9      9        528212       Aaron Lennox             206592
10    10        101042      Aaron Meijers             188621
# ... with more rows, and 3 more variables: birthday <chr>, height <dbl>,
#   weight <int>

In [7]:
teamDF <- collect(team)
playerDF <- collect(player)

In [8]:
teamDF

id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK
5,9984,1750,KSV Cercle Brugge,CEB
6,8635,229,RSC Anderlecht,AND
7,9991,674,KAA Gent,GEN
8,9998,1747,RAEC Mons,MON
9,7947,NA,FCV Dender EH,DEN
10,9985,232,Standard de Liège,STL


In [9]:
player_attr <- tbl(soccer_db, "Player_Attributes")
player_attr_df <- collect(player_attr, n = Inf)

In [10]:
# adding overall ratings to player

data1 <- player %>% select(player_api_id, player_name)

data2 <- player_attr %>% select(player_api_id, overall_rating)

player_final <- data1 %>% left_join(data2)

Joining, by = "player_api_id"


In [11]:
player_attr_df <- player_attr_df %>% mutate(gk_rating=gk_diving + gk_handling + gk_kicking + gk_positioning + gk_reflexes)


In [12]:
data1 <-  player_attr_df %>% select(player_api_id, gk_rating)

In [13]:
data1

player_api_id,gk_rating
505942,43
505942,43
505942,43
505942,38
505942,38
155782,51
155782,51
155782,51
155782,51
155782,51


In [14]:
data1 <- data1 %>% group_by(player_api_id) %>% summarise_each(funs(max))

In [15]:
player_final_df <- collect(player_final, n = Inf)
player_final_df

player_api_id,player_name,overall_rating
505942,Aaron Appindangoye,61
505942,Aaron Appindangoye,61
505942,Aaron Appindangoye,62
505942,Aaron Appindangoye,67
505942,Aaron Appindangoye,67
155782,Aaron Cresswell,47
155782,Aaron Cresswell,51
155782,Aaron Cresswell,52
155782,Aaron Cresswell,53
155782,Aaron Cresswell,53


In [16]:
player_final_df <- player_final_df %>% group_by(player_api_id) %>% summarise_each(funs(max))

In [17]:
player_final_df <- data1 %>% left_join(player_final_df)

Joining, by = "player_api_id"


In [18]:
player_final_df

player_api_id,gk_rating,player_name,overall_rating
2625,153,"Patryk Rachwal,18",63
2752,53,Diego Mainz,72
2768,128,Jose Dorado,75
2770,148,Ignacio Gonzalez,74
2790,144,Alberto Rey,73
2796,340,Javier Jimenez,69
2802,146,Pablo Hernandez,82
2805,141,Ruben Perez,76
2857,143,Ivan Perez,72
2862,146,Vicente Sanchez,76


In [26]:
match <- tbl(soccer_db, "Match")

In [27]:
matchDF <- collect(match, n = Inf)

In [21]:
stacked_data <- stack(matchDF, select = c(home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11))

In [22]:
# adding league id to the player frame
data1 <- cbind(matchDF["league_id"], stacked_data)
data1[3] <- NULL

In [23]:
data1 <- data1 %>% group_by(values) %>% summarise_each(funs(n_distinct))

In [24]:
colnames(data1) <- c("player_api_id", "num_leagues")

In [25]:
player_final_df <- data1 %>% left_join(player_final_df)

Joining, by = "player_api_id"


In [26]:
# adding number of appearances to the player frame

colnames(stacked_data) <- c("player_api_id", "player_info")

app_info <- stacked_data %>% group_by(player_api_id) %>% mutate(appearances = n()) 


In [27]:
app_info[2] <- NULL
app_info <- na.omit(app_info)

In [28]:
player_final_df <- app_info %>% left_join(player_final_df)

Joining, by = "player_api_id"


In [29]:
player_final_df

player_api_id,appearances,num_leagues,gk_rating,player_name,overall_rating
39890,8,1,336,Mark Volders,68
38327,59,1,339,Wouter Biebauw,68
95597,63,2,NA,Bertrand Laquait,NA
30934,114,1,348,Boubacar Barry Copa,73
37990,81,1,361,Stijn Stijnen,73
38391,184,1,373,Silvio Proto,75
39153,41,1,316,Cedric Berthelin,65
37900,180,1,352,Sammy Bossuyt,72
36835,95,1,336,Bram Verbist,67
38252,7,1,341,Frederic Herpoel,68


In [33]:
player_final_df <- player_final_df[!duplicated(player_final_df[,c('player_api_id')]),]
write.csv(player_final_df, file = "player.csv")

In [7]:
team <- tbl(soccer_db, "Team")
team_df <- collect(team, n = Inf)

In [8]:
team_final_df <- team_df %>% select(team_api_id,team_long_name)
dim(team_final_df)

[1] 299   2

In [9]:
team_attributes <- tbl(soccer_db, "Team_Attributes")

In [10]:
team_attr_df <- collect(team_attributes, n = Inf)

In [11]:
team1 <- team_attr_df %>% mutate(build_up_play=buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing)

In [12]:
team1 <- na.omit(team1)

In [13]:
team1 <- team1 %>% select(team_api_id, build_up_play)

In [14]:
team1 <- team1 %>% group_by(team_api_id) %>% summarise_each(funs(max))
dim(team1)

[1] 262   2

In [15]:
team2 <- team_attr_df %>% mutate(chance_creation = chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting)

In [16]:
team2 <- team2 %>% select(team_api_id, chance_creation)

In [17]:
team2 <- team2 %>% group_by(team_api_id) %>% summarise_each(funs(max))
dim(team2)

[1] 288   2

In [18]:
team3 <- team_attr_df %>% mutate(defence = defencePressure+defenceAggression+defenceTeamWidth)

In [19]:
team3 <- team3 %>% select(team_api_id, defence)

In [20]:
team3 <- team3 %>% group_by(team_api_id) %>% summarise_each(funs(max))
dim(team3)

[1] 288   2

In [21]:
dim(team_final_df)

[1] 299   2

In [22]:
team_final_df <- team_final_df %>% left_join(team1)
team_final_df <- team_final_df %>% left_join(team2)
team_final_df <- team_final_df %>% left_join(team3)

Joining, by = "team_api_id"
Joining, by = "team_api_id"
Joining, by = "team_api_id"


In [23]:
team_final_df

team_api_id,team_long_name,build_up_play,chance_creation,defence
9987,KRC Genk,148,182,205
9993,Beerschot AC,NA,185,210
10000,SV Zulte-Waregem,147,153,200
9994,Sporting Lokeren,159,154,200
9984,KSV Cercle Brugge,145,200,195
8635,RSC Anderlecht,149,182,190
9991,KAA Gent,142,184,166
9998,RAEC Mons,160,168,138
7947,FCV Dender EH,NA,NA,NA
9985,Standard de Liège,161,182,205


In [28]:
stacked_data1 <- stack(matchDF, select = c(home_team_goal, away_team_goal))

In [29]:
stacked_data2 <- stack(matchDF, select = c(home_team_api_id, away_team_api_id))

In [30]:
data1 <- cbind(stacked_data1, stacked_data2)

In [31]:
data1[4] <- NULL
data1[2] <- NULL

In [32]:
colnames(data1) <- c("scored_goals", "team_api_id")

In [33]:
data1 <- data1 %>% group_by(team_api_id) %>% summarise_each(funs(sum))

In [34]:
data1

team_api_id,scored_goals
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184
2186,235
4049,35
4064,27
4087,180


In [35]:
team_final_df <- team_final_df %>% left_join(data1)

Joining, by = "team_api_id"


In [36]:
team_final_df

team_api_id,team_long_name,build_up_play,chance_creation,defence,scored_goals
9987,KRC Genk,148,182,205,348
9993,Beerschot AC,NA,185,210,174
10000,SV Zulte-Waregem,147,153,200,306
9994,Sporting Lokeren,159,154,200,279
9984,KSV Cercle Brugge,145,200,195,213
8635,RSC Anderlecht,149,182,190,427
9991,KAA Gent,142,184,166,390
9998,RAEC Mons,160,168,138,129
7947,FCV Dender EH,NA,NA,NA,44
9985,Standard de Liège,161,182,205,341


In [37]:
write.csv(team_final_df , file = "team.csv")